# Sampling a posterior

This notebook shows the basic usage of `cogwheel`.

We will choose an event, a waveform approximant and a prior and generate samples from the posterior distribution.

In [ ]:
path_to_cogwheel = '..'

import sys
sys.path.append(path_to_cogwheel)

import pandas as pd

from cogwheel import data
from cogwheel import gw_prior
from cogwheel import likelihood
from cogwheel import sampling
from cogwheel import waveform
from cogwheel.posterior import Posterior

## Event options

For now, we will use one of the preprocessed `EventData` files included:

In [ ]:
eventnames = sorted(path.name.removesuffix('.npz')
                    for path in data.DATADIR.glob('*.npz'))
print('Available eventnames:')
eventnames

## Approximant options

Pick one from the keys below. Approximants with aligned spins require that the prior has aligned spins as well.

In [ ]:
waveform.APPROXIMANTS

## Prior options

For now, we will choose one of the included `Prior` classes:

In [ ]:
for prior_name, prior_class in gw_prior.prior_registry.items():
    print(f'{prior_name}:\n    {prior_class.__doc__}\n'
          .replace('\n    \n', '\n'))

## Instantiating a `Posterior`

In [ ]:
metadata = pd.read_csv(data.DATADIR/'events_metadata.csv', index_col=0)  # Chirp mass guesses

In [ ]:
# Choose from the above options:
eventname = eventnames[0]
mchirp_guess = metadata['mchirp'][eventname]
approximant = 'IMRPhenomXAS'
prior_name = 'AlignedSpinLVCPriorComovingVT'

post = Posterior.from_event(eventname, mchirp_guess, approximant, prior_name)

Things you might want to double-check at this point:

* Did the maximizer find a high likelihood solution ($\log \mathcal{L} \approx \rm{SNR}^2 / 2$)?
* Is the automatically chosen `mchirp_range` ok?
* Are the default `q_min` and `dt0` ok?

In [ ]:
post.likelihood.lnlike(post.likelihood.par_dic_0)

In [ ]:
post.prior.get_init_dict()

In [ ]:
# Say we want to edit the chirp-mass range:
post.prior = post.prior.reinstantiate(mchirp_range=(20, 50))

<div class="alert alert-block alert-info">
<b>Distance marginalization</b>

You can use a likelihood marginalized semi-analytically over distance to remove this dimension from the sampling (recommended).

Instantiate as below. `post_md.likelihood.lnlike` will no longer have an interpretation as $\rm SNR^2 / 2$.
</div>

In [ ]:
lookup_table = likelihood.LookupTable()
post_md = Posterior.from_event(eventname, mchirp_guess,
                               approximant='IMRPhenomXPHM',
                               prior_class='MarginalizedDistanceIASPrior',
                               likelihood_class=likelihood.MarginalizedDistanceLikelihood,
                               likelihood_kwargs={'lookup_table': lookup_table})

## Instantiating a `Sampler`

The implemented samplers so far are [PyMultiNest](https://johannesbuchner.github.io/PyMultiNest/) and [dynesty](https://dynesty.readthedocs.io/en/stable/).

### PyMultiNest

In [ ]:
pym = sampling.PyMultiNest(post)

You can see and edit the options passed to `pymultinest.run()`. For example, to change the number of live points:

In [ ]:
pym.run_kwargs['n_live_points'] = 1000

### dynesty
Works similarly, except `run_kwargs` now will be keyword arguments to `dynesty.DynamicNestedSampler` or `dynesty.DynamicNestedSampler.run_nested`

In [ ]:
dyn = sampling.Dynesty(post)

### Running the sampler

In [ ]:
parentdir = 'example'  # Directory that will contain parameter estimation runs

In [ ]:
sampler = pym  # or `dyn`
rundir = sampler.get_rundir(parentdir)

You can run the sampler live or, if you have access to a cluster, submit a job to the workload manager.

In [ ]:
# Run "live" (will take a while):
sampler.run(rundir)

In [ ]:
# Alternatively, submit job to a scheduler (SLURM or LSF implemented)
sampler.submit_slurm(rundir)